In [1]:
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
%pylab inline
import xgboost
import pickle

Populating the interactive namespace from numpy and matplotlib


/Users/axa/.pyenv/versions/2.7.12/envs/DS/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/Users/axa/.pyenv/versions/2.7.12/envs/DS/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
test = pd.read_csv('./data/test.csv')

#### Create features on age

In [3]:
# exist a class containing missings
labels = ["child", "adult", "grandpa"]
test['age_group'] = pd.cut(test['Age'], bins=[0, 18, 60, inf], right=False, labels=labels)

#### Create features on family size

In [4]:
# Create a variable : family size = sibilings + Parents + the passenger
test['family_size'] = test['SibSp']+test['Parch']+1

In [5]:
labels_family = ["alone", "small_family", "large_family"]
test['family_label'] = pd.cut(test['family_size'], bins=[1,2,4,inf],
                               right=False, labels=labels_family)

####  Features on title

In [6]:
# Create a variable Title
test['title'] = test['Name'].apply(lambda x: x.split(', ')[1].split('.')[0])

In [7]:
master_index = np.where(test['title']=='Master')[0]
miss_index = np.where(test['title']=='Miss')[0]
mr_index = np.where(test['title']=='Mr')[0]
mrs_index = np.where(test['title']=='Mrs')[0]

In [8]:
# Initialize tite_group
test['title_group'] = 'other_title'

In [9]:
test.ix[master_index,'title_group'] = 'Master'
test.ix[miss_index,'title_group'] = 'Miss'
test.ix[mr_index,'title_group'] = 'Mr'
test.ix[mrs_index,'title_group'] = 'Mrs'

#### Get the cabine location

In [10]:
filled_cabine = test['Cabin'].notnull()

In [11]:
test["cabine_categ"] = NaN

In [12]:
# Get the first letter of the cabine
test.ix[filled_cabine, 'cabine_categ'] = test[filled_cabine].Cabin.\
                                            apply(lambda x: x[0])

#### Create dummy

In [20]:
test_min = test[['PassengerId','SibSp','Parch']]

In [21]:
dummies_class = pd.get_dummies(test['Pclass'],prefix='class_')
dummies_embarked = pd.get_dummies(test['Embarked'],prefix='embarked_',dummy_na=True)
dummies_age = pd.get_dummies(test['age_group'],prefix='age_',dummy_na=True)

dummies_sex = pd.get_dummies(test['Sex'])
dummies_family_label = pd.get_dummies(test['family_label'])
dummies_title_group = pd.get_dummies(test['title_group'])

In [22]:
test_featured = test_min.join(dummies_class).join(dummies_embarked).join(dummies_age).\
            join(dummies_sex).join(dummies_family_label).join(dummies_title_group)

#### Apply model

In [35]:
with open('./model_logistic.model', 'r') as model:
    model = pickle.load(model)

In [36]:
column_to_use = list(test_featured.columns)
column_to_use.remove('PassengerId')

In [38]:
prediction = pd.DataFrame(model.predict(test_featured[column_to_use]),columns =['Survived'])

In [39]:
final = pd.DataFrame(test_featured['PassengerId']).join(prediction)

In [40]:
# export results
final.to_csv('./Submission.csv',index= False)